# Mergin Data: Types of Merges

In this notebook I'll showcase three types of merges with Pandas:

- One-to-one
- Many-to-one
- Many-to-Many

All these merges can be performed using the same Pandas function: [pd.merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

    DataFrame.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None) → 'DataFrame'

In [2]:
# import required library: Pandas
import pandas as pd
import numpy as np
# I'll access world bank data using pandas_datareader
import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
import requests

/home/fede/anaconda3/envs/geopandas/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
# create a list of all country codes, excluding aggregates
country_codes = wb.get_countries()
country_codes = country_codes[country_codes['region'] != 'Aggregates'] #exclude aggregates
country_codes.head(3)

,iso3c,iso2c,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.51670
1,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.52280
3,AGO,AO,Angola,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,IBRD,Luanda,13.2420,-8.81155


In [7]:
print(country_codes['iso2c'].values)


['AW' 'AF' 'AO' 'AL' 'AD' 'AE' 'AR' 'AM' 'AS' 'AG' 'AU' 'AT' 'AZ' 'BI'
 'BE' 'BJ' 'BF' 'BD' 'BG' 'BH' 'BS' 'BA' 'BY' 'BZ' 'BM' 'BO' 'BR' 'BB'
 'BN' 'BT' 'BW' 'CF' 'CA' 'CH' 'JG' 'CL' 'CN' 'CI' 'CM' 'CD' 'CG' 'CO'
 'KM' 'CV' 'CR' 'CU' 'CW' 'KY' 'CY' 'CZ' 'DE' 'DJ' 'DM' 'DK' 'DO' 'DZ'
 'EC' 'EG' 'ER' 'ES' 'EE' 'ET' 'FI' 'FJ' 'FR' 'FO' 'FM' 'GA' 'GB' 'GE'
 'GH' 'GI' 'GN' 'GM' 'GW' 'GQ' 'GR' 'GD' 'GL' 'GT' 'GU' 'GY' 'HK' 'HN'
 'HR' 'HT' 'HU' 'ID' 'IM' 'IN' 'IE' 'IR' 'IQ' 'IS' 'IL' 'IT' 'JM' 'JO'
 'JP' 'KZ' 'KE' 'KG' 'KH' 'KI' 'KN' 'KR' 'KW' 'LA' 'LB' 'LR' 'LY' 'LC'
 'LI' 'LK' 'LS' 'LT' 'LU' 'LV' 'MO' 'MF' 'MA' 'MC' 'MD' 'MG' 'MV' 'MX'
 'MH' 'MK' 'ML' 'MT' 'MM' 'ME' 'MN' 'MP' 'MZ' 'MR' 'MU' 'MW' 'MY' 'NA'
 'NC' 'NE' 'NG' 'NI' 'NL' 'NO' 'NP' 'NR' 'NZ' 'OM' 'PK' 'PA' 'PE' 'PH'
 'PW' 'PG' 'PL' 'PR' 'KP' 'PT' 'PY' 'PS' 'PF' 'QA' 'RO' 'RU' 'RW' 'SA'
 'SD' 'SN' 'SG' 'SB' 'SL' 'SV' 'SM' 'SO' 'RS' 'SS' 'ST' 'SR' 'SK' 'SI'
 'SE' 'SZ' 'SX' 'SC' 'SY' 'TC' 'TD' 'TG' 'TH' 'TJ' 'TM' 'TL' 'TO' 'TT'
 'TN' 

In [42]:
# I'll get a list of country names with populations, from the World Bank
# Using pandas datareader
# see https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#world-bank

pop = (
        wb.download(indicator='SP.POP.TOTL',country=country_codes['iso2c'], start=2015, end=2015)
       .reset_index()
       .drop(columns='year')
       .rename(columns={'SP.POP.TOTL':'population'})
       #.sort_values(by='population', ascending=False) 
      )
pop.head()

/home/fede/anaconda3/envs/geopandas/lib/python3.7/site-packages/pandas_datareader/wb.py:593: UserWarning: Non-standard ISO country codes: JG, XK
  "Non-standard ISO " "country codes: %s" % tmp, UserWarning


,country,population
0,Aruba,104341.0
1,Afghanistan,34413603.0
2,Angola,27884381.0
3,Albania,2880703.0
4,Andorra,78011.0


In [4]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q.AU.PXP_IX.?startPeriod=1957&endPeriod=2016'

# Get data from the above URL using the requests package
data = requests.get(url).json()

# Load data into a pandas dataframe
auxp = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])

# Show the last five observiations
auxp.tail()

,@TIME_PERIOD,@OBS_VALUE
223,2015-Q4,85.5465473860777
224,2016-Q1,81.5208275090858
225,2016-Q2,82.6390830304725
226,2016-Q3,85.9938495946324
227,2016-Q4,98.6301369863014


symbols (string) – Possible formats: 1. DB/SYM: The Quandl ‘codes’: DB is the database name, SYM is a ticker-symbol-like Quandl abbreviation for a particular security. 2. SYM.CC: SYM is the same symbol and CC is an ISO country code, will try to map to the best single Quandl database for that country. Beware of ambiguous symbols (different securities per country)! Note: Cannot use more than a single string because of the inflexible way the URL is composed of url and _get_params in the superclass  


from: https://www.quandl.com/data/ODA-IMF-Cross-Country-Macroeconomic-Statistics/documentation

Data Organization

The quickest way to find a dataset within the IMF database is via search. Click the Data tab on the left of this page and then type your query (including both indicator name and country name) into the search box marked "search this database".

All IMF datasets can also be accessed directly via their unique Quandl code. The codes for these datasets follow the format ODA/{COUNTRY}_{INDICATOR}.

For example, the Quandl code for population of Albania is ODA/ALB_LP, where ALB is the ISO code for Albania and LP is the indicator code for population. The table below lists all available indicators; note that not all indicators are available for all countries. You can see a list of all country ISO codes here.

In [48]:
# symbol = 'WIKI/AAPL' Inflation % change, Average Consumer Prices
symbol = 'ODA/ABW_PCPIPCH'
temp = web.DataReader(symbol, 'quandl', '2019-01-01', '2020-01-01', api_key='FBwSgrKW14w3TMa6a8un')
temp

,Value
Date,
2019-12-31,1.377


In [49]:
temp.iloc[0,0]

1.3769999999999998

In [52]:
d = pd.Series(dtype='float') # initialize pandas series
print(len(country_codes))
for country_code in country_codes['iso3c']:
    print(np.where(country_codes['iso3c']==country_code))
    try:
        d[country_code] = web.DataReader(f'ODA/{country_code}_PCPIPCH', 'quandl', '2019-01-01', '2020-01-01', api_key='FBwSgrKW14w3TMa6a8un').iloc[0,0]
    except:
        d[country_code] = np.nan

218
(array([0]),)
(array([1]),)
(array([2]),)
(array([3]),)
(array([4]),)
(array([5]),)
(array([6]),)
(array([7]),)
(array([8]),)
(array([9]),)
(array([10]),)
(array([11]),)
(array([12]),)
(array([13]),)
(array([14]),)
(array([15]),)
(array([16]),)
(array([17]),)
(array([18]),)
(array([19]),)
(array([20]),)
(array([21]),)
(array([22]),)
(array([23]),)
(array([24]),)
(array([25]),)
(array([26]),)
(array([27]),)
(array([28]),)
(array([29]),)
(array([30]),)
(array([31]),)
(array([32]),)
(array([33]),)
(array([34]),)
(array([35]),)
(array([36]),)
(array([37]),)
(array([38]),)
(array([39]),)
(array([40]),)
(array([41]),)
(array([42]),)
(array([43]),)
(array([44]),)
(array([45]),)
(array([46]),)
(array([47]),)
(array([48]),)
(array([49]),)
(array([50]),)
(array([51]),)
(array([52]),)
(array([53]),)
(array([54]),)
(array([55]),)
(array([56]),)
(array([57]),)
(array([58]),)
(array([59]),)
(array([60]),)
(array([61]),)
(array([62]),)
(array([63]),)
(array([64]),)
(array([65]),)
(array([66]),)
(

In [57]:
inflation_df = pd.DataFrame(d)
inflation_df.columns = ['inflation']
inflation_df.head(5)

,inflation
ABW,1.377
AFG,1.900
AGO,17.463
ALB,1.999
AND,NaN


## One-to-one Merge

In this type of merge, there should be no duplicate values in each column. If there are duplicate values, the duplicate value will be repeated. 

In [25]:
pop.head()

,country,population
0,Aruba,104341.0
1,Afghanistan,34413603.0
2,Angola,27884381.0
3,Albania,2880703.0
4,Andorra,78011.0


In [26]:
country_codes.head()

,iso3c,iso2c,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.51670
1,AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.52280
3,AGO,AO,Angola,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,IBRD,Luanda,13.2420,-8.81155
4,ALB,AL,Albania,Europe & Central Asia,Europe & Central Asia (excluding high income),Upper middle income,IBRD,Tirane,19.8172,41.33170
5,AND,AD,Andorra,Europe & Central Asia,,High income,Not classified,Andorra la Vella,1.5218,42.50750


In [43]:
# first we will merge the population data with the country codes data, this is a one-to-one merge
pop2 = pop.merge(country_codes[['name','iso3c']], left_on='country', right_on='name')
pop2.drop(columns=['name'], inplace=True)
pop2.head(3)

,country,population,iso3c
0,Aruba,104341.0,ABW
1,Afghanistan,34413603.0,AFG
2,Angola,27884381.0,AGO


In [44]:
print(len(pop))
print(len(pop2))

217
217


Now, I'll merge pop2 with inflation

In [60]:
pop3 = pop2.merge(inflation_df, left_on='iso3c', right_index=True)
pop3.head(3)

,country,population,iso3c,inflation
0,Aruba,104341.0,ABW,1.377
1,Afghanistan,34413603.0,AFG,1.900
2,Angola,27884381.0,AGO,17.463


In [61]:
print(len(pop3))

217


## Many-toone

In this type of merge, 